In [1]:
%reload_ext dotenv
%dotenv

In [35]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
import os
import openai
import pandas as pd

def get_query_engine_from_text(txtfile):
  openai.api_key = os.environ['OPENAI_API_KEY']
  documents = SimpleDirectoryReader(txtfile).load_data()
  index = VectorStoreIndex.from_documents(documents)
  query_engine = index.as_query_engine()
  return query_engine

def csv_to_llminput(csvpath,txtOutputPath):
    df = pd.read_csv(csvpath)
    f = open(txtOutputPath,"w")
    print("Started writing to: ",txtOutputPath)
    for index, row in df.iterrows():
      f.write("Price of house with area of {area} and {bedrooms} bedrooms is {price}\n".format(area=row['area'],bedrooms=row['bedrooms'],price=row['price']))
    f.close()
    print("Completed writing to: ",txtOutputPath)

In [38]:
csv_to_llminput('./data/Housing.csv','./data/Housing-csv-llminput.txt')

Started writing to:  ./data/Housing-csv-llminput.txt
Completed writing to:  ./data/Housing-csv-llminput.txt


In [40]:
query_engine = get_query_engine_from_text('data')
resp = query_engine.query("What house has least number of bedrooms?")

resp.response

'\nThe house with the least number of bedrooms is a house with an area of 3970 and 1 bedroom, with a price of 2275000.'

In [10]:
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI

llm = OpenAI(api_token=os.environ['OPENAI_API_KEY'])
pandas_ai = PandasAI(llm,enable_cache=False)

df = pd.read_csv('./data/Housing.csv')
pandas_ai.run(df,prompt='convert dataframe to JSON')


Traceback (most recent call last):
  File "/home/shreyakqss/quantitative-llm/venv/lib/python3.9/site-packages/pandasai/smart_datalake/__init__.py", line 292, in chat
    result = self._code_manager.execute_code(
  File "/home/shreyakqss/quantitative-llm/venv/lib/python3.9/site-packages/pandasai/helpers/code_manager.py", line 199, in execute_code
    return analyze_data(self._get_original_dfs())
  File "<string>", line 2, in analyze_data
NameError: name 'json_data' is not defined


KeyboardInterrupt: 